<h1>OpenStreetMap Data Wrangling using Python and SQL</h1>

<h2>Map Area</h2>

<h3>Albany, NY, United States</h3>

http://www.openstreetmap.org/export#map=11/42.7805/-73.8501 

I chose this city, as I have been living in this area for last five years and would like an opportunity to contribute to its improvement in the openstreetmap.org project. I have chosen to explore the suburbs of Albany, NY rather than the downtown area.

<h2>Problems encountered in the map</h2>

After downloading the dataset and running it against a provisional data.py file, I noticed the following main problems with the dataset, as follows:

<ul>
  <li>Abbreviated and nonuniform street names ('St.', 'St', 'street', 'Ave', 'Ave.' etc).</li>
  <li>Incomplete street names (Sparrowbush, Deltour etc.).</li>
  <li>Inconsistent and incorrectly entered postal codes (12303-9998, 12180-8368 etc.).</li>
  <li>Incorrect city names (Schenectary, Rexrford etc.).</li>
</ul>  

<h3>Abbreviated and incomplete street names</h3>

The osm file revealed basic street name inconsistencies caused due to abbreviations or use of small case letters. Some street names were incorrectly entered and were missing street type, for example Sparrowbush instead of Sparrowbush Road. An attempt was made in audit.py to fix these problems by replacing "St." with "Street", "road" with "Road", "rt" with "Route" etc. 

A list of expected street types like Street, Avenue, Circle etc. was created which do not need to be cleaned. If the last word of a street type was not in the expected list, they were stored in a separate dictionary. It gave an idea about what inconsistent street types are present in the data. The osm file was scanned for tags that were street names (type="addr:street") and the old abbreviated street name was mapped to a new better one using a mapping dictionary. A snippet of the code to update street name is given below.

In [7]:
def update_name(name, mapping):    
    n = street_type_re.search(name) #get the street type
    if n:
        n = n.group()
    for m in mapping: # check if a better name exists for a street type in mapping dict
        if n == m:
            name = name[:-len(n)] + mapping[m] #replace old name with new
    return name

<h3>Incorrect postal codes</h3>

Most postal codes in the Capital District region are all five digits for this map, but some are 9 digits having trailing digits (12180-8368) after the first five digits. Wrote some code to drop the trailing digits and the  '-' after the five digit postal code. 

One particular postal code 1220y stood out as incorrectly entered. I needed to find out what address it belonged to in order to replace it with the correct code. It belonged to<i> 1, State Street, Albany</i> which has zip code 12207.

In [11]:
def update_postcode(postcode):   #update postal code to remove trailing digits
    if len(postcode)==10 and postcode[5] == '-':
        return postcode[:5]
    elif postcode == "1220y":# update the one instance of incorrectly entered postal code for this extract
        return "12207"
    else:
        return postcode

<h3>Incorrect City Names</h3>

While most city names in this dataset were fine, some were incorrectly spelled. Like Schenectary instead of Schenectady. 

In [9]:
def update_city(name, mapping_city):   
    for m in mapping_city:        #replace incorrect name with correct one from mapping dict
        if name == m:            
            name = mapping_city[m]
    return name

<h2>Data Overview and Statistics</h2>

The data from the OSM XML file was converted to tabular form which could be written into CSV files. These CSV files could easily be imported to SQL tables.

<h3>File Sizes</h3>

In [10]:
import os
print('The albany.osm file is {} MB'.format(os.path.getsize('albany.osm')/1.0e6))
print('The osm.db file is {} MB'.format(os.path.getsize('osm.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6))

The albany.osm file is 95.825419 MB
The osm.db file is 67.355648 MB
The nodes.csv file is 34.323738 MB
The nodes_tags.csv file is 2.893896 MB
The ways.csv file is 2.874296 MB
The ways_tags.csv is 8.955983 MB
The ways_nodes.csv is 11.009704 MB


<h3>Number of nodes</h3>

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;sqlite> SELECT COUNT(*) FROM nodes;<br /><br /></p>


397468

<h3>Number of ways</h3>


<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;sqlite> SELECT COUNT(*) FROM ways;<br /><br /></p>


46401

<h3>Number of unique contributing users</h3>

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;
sqlite> SELECT COUNT(DISTINCT(e.uid))<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   ...> FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;
<br /><br /></p>   

556

<h3>Top contributing users</h3>

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp; SELECT e.user, COUNT(*) as num<br />
&nbsp;&nbsp; FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e<br />
&nbsp;&nbsp;    GROUP BY e.user<br />
&nbsp;&nbsp; ORDER BY num DESC<br /><br /></p>

"nfgusedautoparts"	"101654"<br />
"woodpeck_fixbot"	"97147"<br />
"JessAk71"	"50979"<br />
"ke9tv"	"35085"<br />
"KindredCoda"	"21036"<br />
"RussNelson"	"15605"<br />
"Юкатан"	"12697"<br />
"eugenebata"	"6639"<br />
"bmcbride"	"6056"<br />
"EdSS"	"5713"<br />

<h3>Users whose posts appear only once</h3>

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp; sqlite> SELECT COUNT(*)<br />
&nbsp;&nbsp;    ...> FROM<br />
&nbsp;&nbsp;    ...>     (SELECT e.user, COUNT(*) as num<br />
&nbsp;&nbsp;    ...>      FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e<br />
&nbsp;&nbsp;    ...>      GROUP BY e.user<br />
&nbsp;&nbsp;    ...>      HAVING num=1)  u;<br /><br /></p>

116

<h2>Additional Data Exploration</h2>

<h3>Most Popular Cuisines</h3>

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;sqlite> SELECT tags.value, COUNT(*) as num<br />
&nbsp;&nbsp;&nbsp;&nbsp;   FROM (SELECT * FROM nodes_tags<br />
&nbsp;&nbsp;&nbsp;&nbsp;   UNION ALL<br />
&nbsp;&nbsp;&nbsp;&nbsp;   SELECT * FROM ways_tags) tags<br />
&nbsp;&nbsp;&nbsp;&nbsp;   WHERE tags.key = 'cuisine'<br />
&nbsp;&nbsp;&nbsp;&nbsp;   GROUP BY tags.value<br />
&nbsp;&nbsp;&nbsp;&nbsp;   ORDER BY num DESC LIMIT 10;<br /><br /></p>

pizza,74 <br />
sandwich,63<br />
burger,62<br />
italian,39<br />
chinese,33<br />
mexican,23<br />
coffee_shop,16<br />
donut,16<br />
diner,13<br />
american,12<br />

<h3>Most popular pizza places</h3>

Since pizza is one of the most popular food here, which are the places serving pizza?

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;SELECT tags.value, COUNT(*) as total_num<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FROM (<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SELECT * FROM nodes_tags <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = "pizza") i ON nodes_tags.id = i.id  <br />    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;UNION ALL<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SELECT * FROM ways_tags<br /> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value = "pizza") j ON ways_tags.id = j.id<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;) tags<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;WHERE tags.key = 'name'<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GROUP BY tags.value<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ORDER BY total_num DESC LIMIT 10;<br /><br /></p>

"I Love NY Pizza",3<br />
"Papa John's",3<br />
"Pizza Hut",3<br />
"Domino's Pizza",2<br />
Dominos,2<br />
"Paesan's Pizza",2<br />
"A J's Pizzeria",1<br />
"Bacchus Woodfired",1<br />
"Big Guys Pizzeria",1<br />
"Chef's Takeout Restaurant",1<br />

<h3>Top Fast Food Chains</h3>

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;SELECT tags.value, COUNT(*) as total_num<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FROM (SELECT * FROM nodes_tags <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN (SELECT DISTINCT(id) FROM<br /> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;nodes_tags WHERE value = "fast_food") i ON nodes_tags.id = i.id<br />         
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;UNION ALL<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SELECT * FROM ways_tags <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value = "fast_food") j ON ways_tags.id = j.id <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;) tags<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;WHERE tags.key = 'name'<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GROUP BY tags.value<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ORDER BY total_num DESC LIMIT 10;<br /><br /></p>
    

"Dunkin' Donuts",33<br />
"McDonald's",27<br />
Subway,24<br />
"Burger King",13<br />
"Subway Subs",10<br />
"Wendy's",10<br />
"Mr Subb",8<br />
"Taco Bell",6<br />
"Bruegger's Bagels",4<br />
"Moe's Southwest Grill",4<br />

<h3>Number of Stewart's' Shops</h3>

Stewart's is a very popular convenience store in this area. How many Stewart's stores are there in the map?

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;SELECT COUNT(*) FROM <br />
&nbsp;&nbsp;(SELECT * FROM nodes_tags WHERE value LIKE "Stewart's"<br /> 
&nbsp;&nbsp;UNION ALL <br />
&nbsp;&nbsp;SELECT * FROM ways_tags WHERE value LIKE "Stewart's" );<br /><br /></p>

121

How are they spread across the region? What cities have most shops?

<p style="background-color:#F1EDED;"><br />&nbsp;&nbsp;SELECT tags.value, COUNT(*) as total_num<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FROM (SELECT * FROM nodes_tags <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value LIKE "Stewart's") i ON nodes_tags.id = i.id      <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;UNION ALL<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SELECT * FROM ways_tags<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value LIKE "Stewart's") j ON ways_tags.id = j.id <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;) tags<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;WHERE tags.key = 'city'<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GROUP BY tags.value<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ORDER BY total_num DESC<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;LIMIT 5;<br /><br /></p>

Albany,13<br />
Schenectady,11<br />
Troy,9<br />
Scotia,4<br />
Altamont,3<br />

<h3>Analysing timestamps</h3>

Since the timestamps for all data entries are available in both nodes and ways tags, I decided to take a look if the map has been kept up to date by volunteers or it needs more contribution? The following bar graph shows the frequency of entries by year.

<img src="images/plot.png">

The figure shows that while 2009 has maximum entries, for the consecutive years the map has been consistently updated too, until recently in October 2017.

<h2>Conclusion</h2>

The data for Albany, NY area was cleaned and street names, postal codes and city data was validated for accuracy. Some basic statistics about the data were obtained and additional exploration about popular cuisines, pizza places, fast food chains etc. was done.

<h3>Additional Ideas for improving OSM</h3>

After this review, it's seen that the data for Albany area is incomplete, though I believe it's been well cleaned for this exercise. Some additional ideas that I came across which can be implemented:

<ul>
  <li>While looking at Stewart Shops in the area, I found that some shops were missing the city field. This also seems to be the case for other entries. Data entries should be validated for completeness.</li>
  <li>Subway and Subway Subs point to the same chain but are listed separately. This data can be cleaned up.</li>
  <li>Pizza Hut is listed as both pizza and fast food. This information should be standardized.</li>
  <li>The timestamps of entries can be further analysed to find out how recent the data is, and how frequently it is being contributed to. This could help contributors to find where there is most need of updating the data.</li>
</ul>  

<h3>Benefits and anticipated problems</h3>

Standardizing the data on openstreetmap.org and validating it for correctness and completeness will make the data more useful and accessible to customers, increasing it's popularity too. It will bring it at par with other popular map services like Google or Bing Maps which in turn will help improve the data, as more people would contribute to the project. 

But since OSM is run entirely by volunteers, doing this could be more challenging. For example, it may not be practical to physically verify latest location information. Or to obtain missing information. 

Implementing some methods of cross validation might help. So would encouraging creation of better bots/scripts to import data to OSM.

